In [1]:
import tensorflow as tf
import numpy as np
import os
 
def get_files(path_pos,path_neg,label_pos,label_neg):
    TC = []
    label_TC = []
    nonTC = []
    label_nonTC = []
    # data loader
    file_dir_TC=path_pos
    file_dir_nonTC=path_neg
    #len_valTC=len(os.listdir(file_dir_TC))
    for file in os.listdir(file_dir_TC):
        name = file.split('_')
        if name[0] == label_pos:
            TC.append(file_dir_TC + file)
            label_TC.append(1)
    #for file in os.listdir(file_dir_nonTC)[:len_valTC]:
    for file in os.listdir(file_dir_nonTC):
        name = file.split('_')
        if name[0] == label_neg:
            nonTC.append(file_dir_nonTC + file)
            label_nonTC.append(0)
    print("There are %d TC\nThere are %d nonTC" % (len(TC), len(nonTC)))
 
    # shuffle
    image_list = np.hstack((TC, nonTC))
    label_list = np.hstack((label_TC, label_nonTC))
    temp = np.array([image_list, label_list])
    temp = temp.transpose()    
    np.random.shuffle(temp)
 
    image_list = list(temp[:, 0])
    label_list = list(temp[:, 1])
    label_list = [int(i) for i in label_list]
 
    return image_list, label_list
 
# img_list,label_list = get_files(file_dir)
 
# batch
def get_batch(image, label, image_W, image_H, batch_size, capacity):   
    image = tf.cast(image, tf.string)
    label = tf.cast(label, tf.int32)
 
    # queue
    input_queue = tf.train.slice_input_producer([image, label])
 
    image_contents = tf.read_file(input_queue[0])
    label = input_queue[1]
    image = tf.image.decode_jpeg(image_contents, channels=1)
 
    # resize
    image = tf.image.resize_images(image, [image_H, image_W], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.cast(image, tf.float32)
    # image = tf.image.per_image_standardization(image)  
    image_batch, label_batch = tf.train.batch([image, label],
                                              batch_size=batch_size,
                                              num_threads=64,  
                                              capacity=capacity)
  
    return image_batch, label_batch

In [2]:
import tensorflow as tf
def inference(images, batch_size, n_classes):
 
 
    with tf.variable_scope('conv1') as scope:
        weights = tf.get_variable('weights',
                                  shape=[3, 3, 1, 16],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[16],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(images, weights, strides=[1, 1, 1, 1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
 
    with tf.variable_scope('pooling1_lrn') as scope:
            pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pooling1')
            norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
 
    with tf.variable_scope('conv2') as scope:
                weights = tf.get_variable('weights',
                                          shape=[3, 3, 16, 16],
                                          dtype=tf.float32,
                                          initializer=tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
                biases = tf.get_variable('biases',
                                         shape=[16],
                                         dtype=tf.float32,
                                         initializer=tf.constant_initializer(0.1))
                conv = tf.nn.conv2d(norm1, weights, strides=[1, 1, 1, 1], padding='SAME')
                pre_activation = tf.nn.bias_add(conv, biases)
                conv2 = tf.nn.relu(pre_activation, name='conv2')
 
    # pool2 and norm2
    with tf.variable_scope('pooling2_lrn') as scope:
        norm2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
        pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME', name='pooling2')
 
    with tf.variable_scope('local3') as scope:
        reshape = tf.reshape(pool2, shape=[batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = tf.get_variable('weights',
                                  shape=[dim, 128],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[128],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
 
    # local4
    with tf.variable_scope('local4') as scope:
        weights = tf.get_variable('weights',
                                  shape=[128, 128],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[128],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name='local4')
 
    # softmax
    with tf.variable_scope('softmax_linear') as scope:
        weights = tf.get_variable('softmax_linear',
                                  shape=[128, n_classes],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[n_classes],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name='softmax_linear')
 
    return softmax_linear
 
 
 
def losses(logits, labels, name):
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits \
            (logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        if name=='train':
            tf.summary.scalar(scope.name + '/train_loss', loss)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_loss', loss)
    return loss
 
def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op
 
def evaluation(logits, labels, name):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_accuracy', accuracy)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_accuracy', accuracy)
    return accuracy

def recall_precision(logits, labels, name):
    logits = tf.cast(logits, tf.int64)
    labels = tf.cast(labels, tf.int64)
    predict = tf.arg_max(logits,1)
    with tf.variable_scope('recall_precision') as scope:
        TP = tf.count_nonzero(predict * labels)
        TN = tf.count_nonzero((predict - 1) * (labels - 1))
        FN = tf.count_nonzero(predict * (labels - 1))
        FP = tf.count_nonzero((predict - 1) * labels)
        precision = tf.divide(TP, TP + FP)
        recall = tf.divide(TP, TP + FN)
        precision = tf.cast(precision, dtype=tf.float64)
        recall = tf.cast(recall, dtype=tf.float64)
        #f1 = 2 * precision * recall / (precision + recall)
        #f1 = tf.cast(f1, dtype=tf.float32)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_precision', precision)
            tf.summary.scalar(scope.name + '/train_recall', recall)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_precision', precision)
            tf.summary.scalar(scope.name + '/valid_recall', recall)
    return precision, recall

In [3]:
##validation
import os
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
N_CLASSES = 2 
IMG_W = 32  # resize
IMG_H = 32
BATCH_SIZE = 512
CAPACITY = 2000
MAX_STEP = 200

file_dir_valTC='/home/ubuntu/data/valTC_cp/'
file_dir_valnonTC='/home/ubuntu/data/valnonTC/'

valid, valid_label = get_files(file_dir_valTC,file_dir_valnonTC,'valTC','valnonTC')
logs_valid_dir='./valid_lenet_balanced_cor'

valid_batch,valid_label_batch=get_batch(valid,
                                valid_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)

#with tf.Graph().as_default():
ckpt_path = './train_lenet/model.ckpt-1674000'

# x = tf.placeholder(tf.float32, [BATCH_SIZE,IMG_W,IMG_H,1])
# y = tf.placeholder(tf.float32, [BATCH_SIZE])
# y = tf.cast(y,tf.int64)

train_logits = inference(valid_batch, BATCH_SIZE, N_CLASSES)
valid__acc = evaluation(train_logits, valid_label_batch, 'valid')
val_recall, val_precision = recall_precision(train_logits, valid_label_batch, 'valid')
saver = tf.train.Saver()
    
with tf.Session() as sess:
    #valid_batch,valid_label_batch=sess.run([valid_batch,valid_label_batch])
    sess.run(tf.global_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#     variables = tf.contrib.framework.get_variables_to_restore()
#     variables_to_resotre = [v for v in varialbes if v.name.split('/')[0]!='softmax_linear']

    saver.restore(sess,ckpt_path)
    print('--------restore done---------')
    summary_op = tf.summary.merge_all() 

    valid_writer = tf.summary.FileWriter(logs_valid_dir, sess.graph)
    
    print('---------validation start----------')
    for step in np.arange(MAX_STEP):  
        try:        
            for step in np.arange(MAX_STEP):
                if coord.should_stop():
                        break
                val_acc_op, val_recall_op, val_precision_op = sess.run([valid__acc, val_recall, val_precision])
                #val_acc, cal_recall, val_precision = sess.run([valid__acc, val_recall, val_precision],feed_dict={x:valid_batch,y:valid_label_batch})
                print('Step %d, valid accuracy = %.2f%%, valid recall = %.2f%%, valid precision = %.2f%%' %(step, val_acc_op*100.0, val_recall_op*100.0, val_precision_op*100.0))

                summary_str = sess.run(summary_op)
                valid_writer.add_summary(summary_str, step)

        except tf.errors.OutOfRangeError:
            print('Done validation -- epoch limit reached')

        finally:
            coord.request_stop()

There are 14355 TC
There are 434488 nonTC
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use `argmax` instead
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from ./train_lenet/model.ckpt-1674000
--------restore done---------
---------validation start----------
Step 0, valid accuracy = 86.33%, valid recall = 65.22%, valid precision = 25.86%
Step 1, valid accuracy = 84.38%, valid recall = 61.11%, valid precision = 24.44%
Step 2, valid accuracy = 86.52%, valid recall = 76.92%, valid precision = 21.28%
Step 3, valid accuracy = 83.40%, valid recall = 70.83%, valid precision = 26.15%
Step 4, valid accuracy = 84.77%, valid recall = 83.33%, valid precision = 19.23%
Step 5, valid accuracy = 87.89%, valid recall = 58.82%, valid precision = 23.26%
Step 6, valid accuracy = 85.16%, v

Step 85, valid accuracy = 88.87%, valid recall = 76.47%, valid precision = 27.66%
Step 86, valid accuracy = 87.89%, valid recall = 62.50%, valid precision = 25.64%
Step 87, valid accuracy = 86.91%, valid recall = 57.14%, valid precision = 27.27%
Step 88, valid accuracy = 87.89%, valid recall = 72.73%, valid precision = 34.04%
Step 89, valid accuracy = 86.52%, valid recall = 70.59%, valid precision = 27.27%
Step 90, valid accuracy = 89.65%, valid recall = 87.50%, valid precision = 35.00%
Step 91, valid accuracy = 86.52%, valid recall = 68.00%, valid precision = 30.36%
Step 92, valid accuracy = 84.57%, valid recall = 44.44%, valid precision = 10.00%
Step 93, valid accuracy = 86.72%, valid recall = 60.00%, valid precision = 19.15%
Step 94, valid accuracy = 85.16%, valid recall = 43.75%, valid precision = 16.67%
Step 95, valid accuracy = 90.23%, valid recall = 58.82%, valid precision = 25.00%
Step 96, valid accuracy = 87.11%, valid recall = 33.33%, valid precision = 12.82%
Step 97, valid a

Step 184, valid accuracy = 86.13%, valid recall = 60.00%, valid precision = 16.07%
Step 185, valid accuracy = 87.70%, valid recall = 68.75%, valid precision = 21.57%
Step 186, valid accuracy = 87.70%, valid recall = 72.73%, valid precision = 35.56%
Step 187, valid accuracy = 88.67%, valid recall = 71.43%, valid precision = 22.73%
Step 188, valid accuracy = 84.38%, valid recall = 60.00%, valid precision = 24.49%
Step 189, valid accuracy = 86.72%, valid recall = 60.00%, valid precision = 12.50%
Step 190, valid accuracy = 83.20%, valid recall = 75.00%, valid precision = 16.67%
Step 191, valid accuracy = 87.11%, valid recall = 60.00%, valid precision = 24.49%
Step 192, valid accuracy = 84.57%, valid recall = 61.11%, valid precision = 22.00%
Step 193, valid accuracy = 85.94%, valid recall = 70.00%, valid precision = 17.07%
Step 194, valid accuracy = 86.72%, valid recall = 65.00%, valid precision = 28.26%
Step 195, valid accuracy = 84.96%, valid recall = 60.00%, valid precision = 22.64%
Step